In [1]:
import math
from db_conn import get_connection
import datetime
import pandas as pd

### Drugs

In [2]:
def unique_detected_drugs():
    _cur = conn.cursor()
    select_sql = """
    SELECT
      DISTINCT drugs.drug_list AS drug_2
    FROM
      (SELECT
         id,
         drug_list
       FROM article_table_sentences_m m,
             UNNEST(STRING_TO_ARRAY(lower(m.check_drug), ' , ')) AS drug_list
      ) AS drugs
    """
    _cur.execute(select_sql)
    return _cur.fetchall()

def get_drugs():
    _cur = conn.cursor()
    select_sql = """select distinct(lower(cui1_str)) from dict_collapsed_final order by lower(cui1_str)"""
    _cur.execute(select_sql)
    return _cur.fetchall()

In [3]:
conn = get_connection()
unique_drugs = list(map(lambda x: x[0].strip(), unique_detected_drugs()))

In [4]:
def check_with_drug_table():
    unique_drugs = set(map(lambda x: x[0].strip(), unique_detected_drugs()))
    dictionary_drugs = set(map(lambda x: x[0].strip(), get_drugs()))
    
    return list(unique_drugs - dictionary_drugs)

In [5]:
check_list = check_with_drug_table()
# pd.Series(check_list).to_csv('./check_list_drug.csv', index=False)

In [6]:
def update_by_ch(key_w, update):
    print(key_w, update)
    
    _cur = conn.cursor()
    _cur.execute(
    """
        SELECT id, sentence, check_drug
        FROM article_table_sentences_m
        WHERE lower(check_drug) like %s
    """, ("%"+key_w.strip()+"%", )
    )
    fetched = _cur.fetchall()
    
    for f in fetched:
        print(f)
        _id = f[0]
        _sen = f[1]
        _check = f[2]
        
        _check = _check.replace(key_w, update)
        print(_check)
        update_sql = """
                UPDATE article_table_sentences_m
                SET check_drug = %s
                WHERE id = %s
            """
        _cur.execute(update_sql, (_check, _id))
        conn.commit()

In [9]:
replace_empty_list = ['-dependent', '-treated', '-reuptake', '-induced', '-based', '-receptor', 'concentrations', 'post-', '-eluting', '-naïve', '-stimulating', \
                         '-equivalent', '-dose-dense', '–placebo', '-inhibiting', '-containing', '-related', '-like', '-boosted', '-binding', '8 × ', '-only', '-pulse', \
                         '-positive', '-confirmed', '-controlled', '-releasing', '-lowering', 'pre-', '≥110', '-independent', '≥5',\
                     'secreting', 'mothers', 'bothersome', "mothers'", 'senegal']
replace_list = {'0digitoxin':'digitoxin', '0oxygen':'oxygen', 'anastrozole1':'anastrozole', '-to-':'-'}
# remove_list = ['non-']

for ch in check_list:
    print(ch)
    to_be = ch
    found_1 = list(filter(lambda x: ch.find(x)>=0, replace_empty_list))
    for f in found_1:
        to_be = to_be.replace(f, '')
    found_2 = list(filter(lambda x: ch.find(x)>=0, replace_list.keys()))
    for f in found_2:
        to_be = to_be.replace(f, replace_list[f])
    
    if ch != to_be:
        update_by_ch(ch, to_be)


nab-paclitaxel
adenosine-diphosphate
gadolinium-diethylenetriaminepentaacetic acid
clopidogrelam
eosinophils
6-sulphatoxymelatonin
bmiltefosine
microalbuminuric
renin-angiotensin-aldosterone
r-bleomycin
heparinoid
ethinyloestradiol
formulation-bedesonide
aldosteronel
ipratropium
procollagen
α1-antitrypsin
dobutamine20
glucoseauc
fluorodeoxyglucose
flutamide—4803
acarbose–metformin
protein c2
abciximabn
amodiaquinesp
paclitaxel100
hypoalbuminaemia
cy55-arginine-glycine-aspartate
radioiodine
tc–sulfur
n-2-mercaptopropionylglycine
tetracyclines
dopaminergic
anti-exenatide
insulin degludecinsulin
renin–angiotensin–aldosterone
ipoprotein cholesterol
artemether-lumefantrine
lipoprotein c
erythropoietins
bloodglucose
8-sulfophenyltheophylline
5-hydroxyindoleacetic acid
heparin–gp2b
ropinirole1
antithrombin
mgperindopril
trastuzumab–docetaxel
tacrolimus—basiliximab
oestrone
epithelium
high-density-lipoprotein cholesterol
oestradiol
endothelium
irinotecan–temozolomide–temsirolimus
inhibitor-ri

In [8]:
unique_drugs = list(map(lambda x: x[0].strip(), unique_detected_drugs()))
unique_drugs

['levamisole',
 'edoxaban',
 'parathyroid hormone',
 'phosphatidylinositol',
 'diethylenetriaminepentaacetic acid',
 'everolimus',
 'ixekizumab',
 'microalbuminuric',
 'abacavir–lamivudine',
 'bisoprolol',
 'amfetamine',
 'insulin lispro',
 'hydroxy-norclobazam',
 'dulaglutide',
 'lisinopril',
 'folic acid',
 'bendamustinem',
 'acidlevamisole',
 'deutetrabenazine',
 'insulin degludecinsulin',
 'chloroquineproguanil',
 'etanercept',
 'capreomycin',
 'alfacalcidol',
 'low-density-lipoprotein c',
 'spironolactone',
 'aliskiren',
 'azacitidine',
 'cyclophosphamide',
 'anastrozole',
 'oxygenator',
 'pomalidomide',
 'nitrous oxide',
 'treprostinil',
 'tegafur',
 'cabozantinib',
 'exenatide',
 'sulfadoxine-pyrimethamine',
 'amiloride',
 'heparinglycoprotein',
 'insulin glargine',
 'salbutamol',
 'emtricitabine',
 'benazepril',
 'hydroxychloroquine',
 'sotalol',
 'non-secretin',
 'albumin',
 'rofecoxib',
 'endothelium',
 'trastuzumab emtansine',
 'goserelin',
 'tetracycline',
 'carboplatin',
 

### Adverse Effects

In [21]:
def unique_detected_ads():
    _cur = conn.cursor()
    select_sql = """
    SELECT
      DISTINCT drugs.drug_list AS drug_2
    FROM
      (SELECT
         id,
         drug_list
       FROM article_table_sentences_m m,
             UNNEST(STRING_TO_ARRAY(lower(m.check_ad), ' , ')) AS drug_list
      ) AS drugs
    """
    _cur.execute(select_sql)
    return _cur.fetchall()

def get_side_effects():
    _cur = conn.cursor()
    select_sql = """select distinct(lower(llt_name)) from meddra_llt_180717 WHERE exclude=0 AND lower(llt_name)>='u' order by lower(llt_name)"""
    _cur.execute(select_sql)
    return _cur.fetchall()

In [17]:
unique_ads = list(map(lambda x: x[0].strip(), unique_detected_ads()))

In [22]:
def check_with_ad_table():
    unique_ad = set(map(lambda x: x[0].strip(), unique_detected_ads()))
    dictionary_ad = set(map(lambda x: x[0].strip(), get_side_effects()))
    
    return list(unique_ad - dictionary_ad)

In [26]:
check_list = check_with_ad_table()
# pd.Series(check_list).to_csv('./check_list_ad.csv', index=False)

In [25]:
check_list

['type 2 diabetes mellitus',
 'cardiac disorder',
 'syncope',
 'prostate cancers',
 'eruption',
 'dilated cardiomyopathy-3',
 "progression of alzheimer's disease",
 'non-psychotic',
 'burns',
 'noncardiac pain',
 'state-trait-anger-expression',
 'body-mass',
 'carcinoid tumours',
 'premature ventricular contractions',
 'psychopathology',
 'tractional retinal detachment',
 'psoralen',
 'alternate-day',
 'peripheral sensory neuropathy',
 'mucositis',
 'exanthemas',
 'non-wheezers',
 'preterm premature rupture of membranes',
 'antiphlogistics',
 'epidural haematoma',
 'pertussis',
 'contracture',
 'infection-related',
 'peptic-ulcer',
 "crohn's",
 'pulmonary tuberculosis',
 'infragenicular',
 'alternative',
 'traumas',
 'intensely',
 'ecg abnormality',
 'lymphohistiocytosis',
 'epinvb→apnoea',
 'typhoid fever',
 'constrained',
 'regards',
 'ruptured cerebral aneurysm',
 'hemivertebrae',
 'meningiomas',
 'breast cancers',
 'pulmonary capillary hemangiomatosis',
 "hodgkin's lymphoma",
 'ana

In [27]:
def update_by_ch_ad(key_w, update):
    print(key_w, update)
    
    _cur = conn.cursor()
    _cur.execute(
    """
        SELECT id, sentence, check_ad
        FROM article_table_sentences_m
        WHERE lower(check_ad) like %s
    """, ("%"+key_w.strip()+"%", )
    )
    fetched = _cur.fetchall()
    
    for f in fetched:
        print(f)
        _id = f[0]
        _sen = f[1]
        _check = f[2]
        
        _check = _check.replace(key_w, update)
        print(_check)
        update_sql = """
                UPDATE article_table_sentences_m
                SET check_ad = %s
                WHERE id = %s
            """
        _cur.execute(update_sql, (_check, _id))
        conn.commit()

In [29]:
replace_empty_list = ['alternate-day', 'alternative', 'intensely', 'regards', 'variates', 'stranger', 'lasting', 'covariate-', 'teacher-related', 'small-scale', 'covariate–adjusted',\
                     'congenitally', 'afterwards', 'suggesting', 'variables', 'scale-2', 'ovarian-like', 'arbiter2', 'listing',\
                     'windows', 'windows”', 'co-existing', 'scale-a', 'encouraged', 'values<1favor', 'hardship', 'multivariable-adjusted',\
                      '“trash', 'rigorous', 'alternated', 'averaged', 'variance;', 'distinguishing', 'high income', "bachelor's", \
                      'teacher', 
                     ]
replace_list = {'infection-related':'infection', 'abortion missed': 'abortion', 'hepatitis and':'hepatitis',\
               'amyloidosis–dutch':'amyloidosis', 'early neonatal deaths':'neonatal deaths', 'polyps≥1':'polyps',\
                '≥1ischemic stroke':'ischemic stroke', 'strain>−18':'strain', 'pressure≥140':'pressure', 'scale-39':'scale'
               }
# remove_list = ['non-']

for ch in check_list:
    print(ch)
    to_be = ch
    found_1 = list(filter(lambda x: ch.find(x)>=0, replace_empty_list))
    for f in found_1:
        to_be = to_be.replace(f, '')
    found_2 = list(filter(lambda x: ch.find(x)>=0, replace_list.keys()))
    for f in found_2:
        to_be = to_be.replace(f, replace_list[f])
    
    if ch != to_be:
        update_by_ch_ad(ch, to_be)

type 2 diabetes mellitus
cardiac disorder
syncope
prostate cancers
eruption
dilated cardiomyopathy-3
progression of alzheimer's disease
non-psychotic
burns
noncardiac pain
state-trait-anger-expression
body-mass
carcinoid tumours
premature ventricular contractions
psychopathology
tractional retinal detachment
psoralen
alternate-day
alternate-day 
(595021, ' Subgroup analysis Severe group is children with FEV1 as % predicted below median (49%) ; moderate is the other children . p values for difference between subgroups: 0·34 for daily rhDNase vs hypertonic saline ; 0·78 for daily vs alternate-day rhDNase. , Severe illness , Daily rhDNase vs n : 22 ', 'alternate-day , hypertonic , values')
 , hypertonic , values
(595020, ' Subgroup analysis Severe group is children with FEV1 as % predicted below median (49%) ; moderate is the other children . p values for difference between subgroups: 0·34 for daily rhDNase vs hypertonic saline ; 0·78 for daily vs alternate-day rhDNase. , Severe illness ,

(1887424, ' Confirmed cases of infection-related microcephaly , By laboratory determination (Zika virus Zika virus laboratory detection was not available to support notification in many cases. , Region of Brazil Southeast : 28 (10·4%) ', 'infection-related')
infection
(1887436, ' Confirmed cases of infection-related microcephaly , By laboratory determination (Zika virus Zika virus laboratory detection was not available to support notification in many cases. , Laboratory only , STORCH† plus Zika , Region of Brazil North : 1 (1·4%) ', 'infection-related')
infection
(1887437, ' Confirmed cases of infection-related microcephaly , By laboratory determination (Zika virus Zika virus laboratory detection was not available to support notification in many cases. , Laboratory only , STORCH† plus Zika , Brazil : 11 (0·6%) ', 'infection-related')
infection
(1887439, ' Confirmed cases of infection-related microcephaly , By laboratory determination (Zika virus Zika virus laboratory detection was not 

(1887440, ' Confirmed cases of infection-related microcephaly , By laboratory determination (Zika virus Zika virus laboratory detection was not available to support notification in many cases. , Imaging and laboratory , Zika virus , Region of Brazil Southeast : 11 (4·1%) ', 'infection-related')
infection
(1887477, ' Confirmed cases of infection-related microcephaly , By means other than laboratory determination , Clinical or epidemiological only , Brazil : 163 (8·4%) ', 'infection-related')
infection
peptic-ulcer
crohn's
pulmonary tuberculosis
infragenicular
alternative
alternative 
(518853, ' Drug / disease ,  Raltitrexed in colorectal cancer , Alternatives : FUFA ', 'alternatives')
s
(1964261, ' South Asia Composite population attributable risk including modified Alternative Healthy Eating Index as T3\u2008+\u2008T2 vs T1 for south Asia , consistent with direction of association . Composite population attributable risk including modified Alternative Healthy Eating Index as T1\u2008+\

(2703113, " Nephropathy , Macroalbuminuria , Alternative risk equations: United Kingdom Prospective Diabetes Study Outcomes Model 2 Calibration: Slope / intercept / χ2 , p value p values <0·05 reflect larger difference between predicted and observed Kaplan-Meier event rates by the Greenwood-D'Agostino-Nam test (see : ·· ", 'alternative , diabetes , values')
 , diabetes , values
(2703154, ' Retinopathy , Photocoagulation or vitrectomy , Alternative risk equations: ACC / AHA PCEs Discrimination: C-statistic : ·· ', 'alternative')

(2703137, " Nephropathy , Macroalbuminuria , renal failure , end-stage renal disease , doubling of serum creatinine , or >20 mL / min per 1·73 m2 decrease in eGFR , Alternative risk equations: United Kingdom Prospective Diabetes Study Outcomes Model 2 Calibration: Slope / intercept / χ2 , p value p values <0·05 reflect larger difference between predicted and observed Kaplan-Meier event rates by the Greenwood-D'Agostino-Nam test (see : ·· ", 'alternative , diabe

(2378323, " chr7:56169419 , Alternative minor allele frequency Sporadic Parkinson's disease (n=517) : 0·048 ", 'alternative')

(2538827, ' polybrominated diphenyl ethers and cryptorchidism , Exposure–outcome relation (sensitivity analyses) : alternative inputs not available to do sensitivity analyses ', 'alternative')

(2538842, ' Dichlorodiphenyltrichloroethane and fibroids , Exposure–outcome relation (sensitivity analyses) : alternative inputs not available to do sensitivity analyses ', 'alternative')

(2538847, ' polybrominated diphenyl ethers and intelligence quotient points loss and intellectual disability , Low-end estimate (US$) : alternative inputs not available to do sensitivity analyses ', 'alternative')

(2538848, ' polybrominated diphenyl ethers and intelligence quotient points loss and intellectual disability , High-end estimate or alternative scenario (US$) : 367 billion and 133 billion ', 'alternative')

(2538853, ' Dichlorodiphenyltrichloroethane and childhood obesity ,

...

(1090054, ' First Author  (Study)   (Ref . #) ,  Randomized Trials Comparing FFR to Angiography , Pijls (FAME) (14) , Relative Outcome Risk , hazard , or odds ratio depending on publication ; however , meta-analysis used published data to compute and summarize relative risk for all studies (see statistical methods section for details) . Composite MACE All rows represent composite major adverse cardiac events (MACE) (death , myocardial infarction [MI] , revascularization) except where noted ; values<1favor fractional flow reserve (FFR) (lower rate of MACE) , while values >1 favor anatomy. : 0.80 (95% CI: 0.62–1.02) ', 'values<1favor')

(1090049, ' First Author  (Study)   (Ref . #) ,  Randomized Trials Comparing FFR to Angiography , Pijls (DEFER) (3) , Relative Outcome Risk , hazard , or odds ratio depending on publication ; however , meta-analysis used published data to compute and summarize relative risk for all studies (see statistical methods section for details) . Composite MACE All

t-cell lymphoma
sudden cardiac death
puerperal sepsis
illness
erythema migrans
body–mass
lipoma
single ventricle
congestive heart failure-bb
cyanotic
typica
headachet
lung infection
normohomocystinemia
bleedings
hepatic failure
hypokinesia-hyperechogenicity
polycystic kidney
arthritis rheumatoid
neonatal convulsions
lipoprotein-cholesterol low-density
scale-29
scale-29 9
courage
culture negative
frontotemporal dementia
chronic hepatitis
heterotetramers
hiv-1-specific
non-anaemic
postoperative nausea
rbbb
gonorrhoeae
macular edema
oppositional defiant disorder
scale–catastrophising
stroke>retinal infarction>transient
atrial tachycardia
fitting
influenza
febrile seizure
unstressed
meningitis-associated
papillomavirus
torsade de pointes
parkinson's disease-day
assessment-insulin resistance
birth asphyxia
seizures>healthy
developmental delay
lacunar infarction
nevus
ganglionectomy
flutter/fibrillation
hypokalaemia
autism spectrum disorder
nodule
lost consciousness
pulmonary aspergillosis
i

diabetic complication
adenomyosis
genital infection fungal
intracardiac
accidental
morbidity-alternative
morbidity-alternative morbidity-
“administration site reactions
myeloid dysplasia
erythrocyte sedimentation rate high
hypotensive
stress-test
transfusion reaction
falciparum malaria
haemangioma
pulmonary carcinomatous
scale-16
humiliating?
atopic
stress–induced
bone metastases
pericardial effusion
hayfever
renal stones
occipital calcifications
dementia with lewy bodies
primers
toxic neuropathy
massoure
noncoronary artery disease
liver metastases
non-infarct-related
schizophrenia relapse
angina attacks
underdosed
surgical site infection
pulseless ventricular tachycardia
subscale
acute coronary syndrome
invasive candidiasis
hachamovitch
complete heart block
extraversion
non-pregnancyassociated
sinus headache
gravid
epiglottitis
taste disturbance
ragusa
tissue injury
pustular rash
stent thrombosis-segment
electrocardiogram changes
interstitial lung disease
pregnancy-induced
lymphoid le

low platelets
canal polyps
reinfection
ovarian cancers
testicular cancer
anorexia
ct abnormalities
antibiotics
hypercontractile
weight-averaged
weight-averaged weight-
1injury
chronic obstructive pulmonary disease exacerbations
non-septic shock
lung disorders
chronic kidney disease stage 5d
corneal melt
acute myeloid leukaemia
gastrointestinal tuberculosis
chlamydial infection
tropical
malignant pleural effusion
suicide attempt
abused
coronary artery disease-drug-eluting
accident-related
mucocarcinoid tumour
respiratory symptom
floaters
duchenne muscular dystrophy
multivariable–adjusted
meningoencephalitis
scar
cytopenias
myocardial fibrosis
apnoeas
hepatomegaly
hemorrhagic stroke
gout1
polycystic kidney disease
pharmacokinetics
central obesity
wounded
exanthematous
non-beverage
presyncope
non-cardiac chest pain
neutropenic sepsis
encephalomyopathy
obesity
infarction nosos
alcohol poisoning
afib
gastric cancer
attempted suicide
keloid scar
stretching
coronary artery disease
petechiae
c

tricuspid insufficiency
pelvic inflammatory diseases
brain infarction
spasmodic torticollis
gadofluorine-m
anomaly congenital
shaking
cleft lip
predischarge
oral fungal infection
echocardiography abnormalities
psychotic disorder
non-ischemia-guided
hypertonic
black spot
pregnant
mycosis
twin pregnancy
pericolic
systemic inflammatory response syndrome
aesthesioneuroblastoma
chromosomal anomalies
anterior myocardial infarction
myocardial necrosis
stralen
hepatitis b
allergic reaction
nose bleeding
ischaemia
steatohepatitis cirrhosis
neutralising
deathheart
paraparesis
leflunomide
muscular dystrophy
parkinson's disease-night
dementias
acroparesthesias
mitral insufficiency
myelodysplastic syndromes
hypervigilant
oppositional
motor deficit
non-insulin-dependent diabetes mellitus
variant-frontotemporal
non-breast-cancer
scale–coping
hyperemia-peripheral
colorectal cancer
covariate
epicardium
haemorrhagic fever
alperovitch
scale–revised
bone disorders
aureusbacteremia
ductal carcinoma in situ